In [5]:
import pathlib
import pandas as pd
import random 
import pickle

BASE_DIR = pathlib.Path().resolve().parent #Check: /Users/kristoffervarslott/Documents/Python.py/AI_API
SMS_SPAM_DIR = pathlib.Path().resolve()    #Check: /Users/kristoffervarslott/Documents/Python.py/AI_API/SMS-SPAM
DATASETS_DIR = BASE_DIR / 'Datasets' #Check: /Users/kristoffervarslott/Documents/Python.py/AI_API/Datasets

ZIPS_DIR = DATASETS_DIR / 'Zips'     #Check: /Users/kristoffervarslott/Documents/Python.py/AI_API/Datasets/Zips
ZIPS_DIR.mkdir(exist_ok = True, parents = True)

#Spam-Classifier folder: START
SPAM_CLASSIFIER_DIR = DATASETS_DIR / 'Spam-Classifier'

SMS_SPAM_DIR = SPAM_CLASSIFIER_DIR / 'Sms-Spam'
SMS_SPAM_DIR.mkdir(exist_ok = True, parents = True)

YOUTUBE_SPAM_DIR = SPAM_CLASSIFIER_DIR / 'Youtube-Spam'
YOUTUBE_SPAM_DIR.mkdir(exist_ok = True, parents = True)
#Spam-Classifier folder : END

#Exports folder: START
EXPORT_DIR = DATASETS_DIR / 'Exports'
EXPORT_DIR.mkdir(exist_ok = True, parents = True)
SPAM_DATASETS_DIR = EXPORT_DIR / 'Spam_Dataset.csv'
METADATA_EXPORT_PATH = EXPORT_DIR / 'Spam-Metadata.pkl'
TOKENIZER_EXPORT_PATH = EXPORT_DIR / 'Spam-Tokenizer.json'
#Exports folder: END

In [39]:
data = {}

#Opening META_EXPORT_PATH datafile called Spam-Metadata.pkl
#USing pickle to load dataset and storing data in dictonary called data.
with open (METADATA_EXPORT_PATH, 'rb') as f:
    data = pickle.load(f)


In [40]:
X_train = data['X_train']
X_test  = data['X_test']
y_train = data['y_train']
y_test  = data['y_test']
max_words = data['max_words']
max_seq_len = data['max_seq_len']
label_legend = data['label_legend']
label_legend_inverted = data['label_legend_inverted']

In [41]:
import json

In [42]:
data_json = {}

#Opening TOKENIZER_EXPORT_PATH datafile called Spam-Tokenizer.json
#USing json to load dataset and storing data in dictonary called data_json.
with open (TOKENIZER_EXPORT_PATH, 'rb') as f:
    data_json = json.load(f)
